# Analysis to support article on cabs vs. apps

Simple notebook listing expenses with cabs and fuel to support [@vilapedro](https://github.com/vilapedro)'s article on cabs vs. apps.

First let's load the main dataset, using only the columns we need:

In [1]:
import pandas as pd
import numpy as np

usecols = (
    'total_net_value',
    'congressperson_name',
    'subquota_number',
    'subquota_description',
    'cnpj_cpf',
    'year',
    'month'
)

dtype={
    'cnpj_cpf': np.str,
    'subquota_number': np.str
}

df = pd.read_csv(
    '../data/2017-03-15-reimbursements.xz',
    usecols=usecols, dtype=dtype
)

As he's writing about expenses srating from June/2014, let's crop our data:

In [2]:
year2014 = df[df.year == 2014]
year2014june = year2014[year2014['month'] >= 6]
reimbursements = year2014june.append(df[df['year'] >= 2015])
reimbursements.year.unique()

array([2014, 2015, 2016, 2017])

And according to his analysis on cabs usage, we're only interested in 9 specific represenatives:

In [3]:
names = (
    'Marcelo Squassoni',
    'Vanderlei Macris',
    'Francisco Floriano',
    'Marcus Vicente',
    'Marcelo Delaroli',
    'Renata Abreu',
    'Alessandro Molon',
    'Chico D’Angelo',
    'Zeca Dirceu'
)
names = tuple(name.upper() for name in names)
deputies = reimbursements[reimbursements.congressperson_name.isin(names)]
deputies.congressperson_name.unique()

array(['VANDERLEI MACRIS', 'ZECA DIRCEU', 'ALESSANDRO MOLON',
       'FRANCISCO FLORIANO', 'MARCUS VICENTE', 'MARCELO SQUASSONI',
       'RENATA ABREU', 'MARCELO DELAROLI'], dtype=object)

Now let's filter only the expenses done with fuel…

In [4]:
fuel = deputies[deputies.subquota_number == '3']
fuel.subquota_description.unique()

array(['Fuels and lubricants'], dtype=object)

…and with cabs:

In [5]:
taxi = deputies[deputies.subquota_number == '122']
taxi.subquota_description.unique()

array(['Taxi, toll and parking'], dtype=object)

Finally let's group expenses month by month to compare — the hypothesis is that expenses with fuel should decrease when expenses with cabs incresase:

In [6]:
def group_by_month(df):
    keys = ('congressperson_name', 'year', 'month')
    return df.groupby(keys)['total_net_value'] \
        .agg([np.sum, len]) \
        .rename(columns={'len': 'expenses'})

In [7]:
grouped_fuel = group_by_month(fuel)
grouped_fuel

sum  expenses
congressperson_name year month                   
ALESSANDRO MOLON    2014 6      1414.47      13.0
                         7        92.00       1.0
                         10      539.88       5.0
                         11      520.93       5.0
                         12      558.49       5.0
                    2015 1       289.58       2.0
                         2       455.15       3.0
                         3       628.15       4.0
                         4       700.16       5.0
                         5       996.15       8.0
                         6      1380.91      11.0
                         7       627.82       6.0
                         8      1109.22       9.0
                         9      1588.55      15.0
                         10     1564.56      12.0
                         11     1494.17      12.0
                         12      437.30       3.0
                    2016 1       884.96       7.0
                         2      1308.27       8.0
                         3       991.47       6.0
                         4       400.00       4.0
                         5       771.95       5.0
                         6       450.02       4.0
                         7      1180.89       8.0
                         8       533.12       3.0
                         10     1380.16       8.0
                         11      850.74       5.0
                         12      474.21       3.0
FRANCISCO FLORIANO  2014 6      4500.00      42.0
                         7      3604.14      38.0
...                                 ...       ...
ZECA DIRCEU         2014 9      3977.93       2.0
                         10     4285.92       2.0
                         11     3130.27       6.0
                         12     4397.95      10.0
                    2015 1      1409.02      11.0
                         2      3089.04       8.0
                         3      1516.49      10.0
                         4      1262.33       8.0
                         5      2268.94       6.0
                         6      2036.92       1.0
                         7      1671.18       7.0
                         8      2507.65       5.0
                         9      3865.14       5.0
                         10     3208.58      13.0
                         11     3414.32      13.0
                         12     6000.00       5.0
                    2016 1      2033.75       4.0
                         2       635.32       1.0
                         3      5086.30      11.0
                         4      2475.15       7.0
                         5      3873.17       4.0
                         6      6000.00       3.0
                         7      2657.15       5.0
                         8      4075.88       8.0
                         9      5508.14      11.0
                         10     2407.70       6.0
                         11     2268.68       8.0
                         12     3190.78       5.0
                    2017 1       200.34       2.0
                         2       237.96       2.0

[201 rows x 2 columns]

In [8]:
grouped_taxi = group_by_month(taxi)
grouped_taxi

sum  expenses
congressperson_name year month                   
ALESSANDRO MOLON    2014 6       593.00      20.0
                         7       612.00      15.0
                         8       348.00       7.0
                         9       107.00       2.0
                         10     1086.00      22.0
                         11     1629.00      41.0
                         12     1365.07      33.0
                    2015 1      2407.00      62.0
                         2      1955.00      53.0
                         3      1619.00      38.0
                         4      2123.00      41.0
                         5      2141.40      37.0
                         6      1834.69      57.0
                         7      1708.40      31.0
                         8      1912.50      46.0
                         9      1364.00      32.0
                         10     1695.00      36.0
                         11     1232.00      30.0
                         12     1619.40      20.0
                    2016 1       930.00      22.0
                         2       254.00       8.0
                         3      1729.40      42.0
                         4       944.00      21.0
                         5      1344.00      24.0
                         6      1626.00      27.0
                         7      1376.52      34.0
                         8      1297.00      37.0
                         9       665.00      12.0
                         10     1277.81      34.0
                         11      878.56      24.0
...                                 ...       ...
ZECA DIRCEU         2014 9       512.00      16.0
                         10      497.00      11.0
                         11     2441.60      84.0
                         12     2021.90      69.0
                    2015 1       782.90      34.0
                         2      2342.10      77.0
                         3      2500.00      79.0
                         4      1037.64      37.0
                         5      2367.35      93.0
                         6      2700.00      96.0
                         7      1443.39      60.0
                         8      1683.71      62.0
                         9      1320.50      49.0
                         10     1655.05      61.0
                         11     1048.30      44.0
                         12     1074.98      53.0
                    2016 1       370.00       8.0
                         2      1144.15      36.0
                         3      2700.00      72.0
                         4      2700.00      66.0
                         5      1664.31      43.0
                         6      1168.69      33.0
                         7      2133.94      49.0
                         8      2700.00      42.0
                         9       924.42      24.0
                         10     1773.62      47.0
                         11     2018.83      52.0
                         12     1135.23      27.0
                    2017 1       310.77       7.0
                         2      1321.48      39.0

[173 rows x 2 columns]

And… let's export some Excel:

In [9]:
# output = pd.ExcelWriter('fuel_vs_cabs.xlsx')
# grouped_fuel.to_excel(output,'Grouped Fuel')
# grouped_taxi.to_excel(output,'Grouped Taxi')
# fuel.to_excel(output,'Fuel')
# output.save()

According to Pedro nothing interesting in this comparison, but let's leave this notebook here if anyone else is interested in this comparison.